# Perform GSEA using GSEAPY  

Following the potocol defined here: https://gseapy.readthedocs.io/en/latest/gseapy_tutorial.html#use-gsea-command-or-gsea


In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina' # mac
import pandas as pd
import gseapy as gp
import numpy as np
import matplotlib.pyplot as plt

In [6]:
%%bash 
jupyter nbconvert --execute  --to notebook --inplace plot_GSEA.ipynb

[NbConvertApp] WARNING | Config option `template_path` not recognized by `NotebookExporter`.
[NbConvertApp] Converting notebook plot_GSEA.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: ir
[NbConvertApp] Writing 41663 bytes to plot_GSEA.ipynb


In [2]:
gp.__version__

'0.9.9'

In [2]:
def run_GSEA_gene(celltype,nperm=1000,rseed=1000,
             rnkfile="../figures/Fig2/subfigs/fig_2.prom.sub_vs_sub.fisher.csv",
             glist='../dat/glists/gsea_final_all.gmt',col=0,
             rank_metrics='odds',**kwargs):
    '''
    col=1 - Z, 0 - logFC
    '''

    gene_exp_alpha = pd.read_csv(rnkfile)[['celltype','gene',rank_metrics]]
            
    #gene_exp_alpha.head()
    rnk = gene_exp_alpha.loc[gene_exp_alpha["celltype"]==celltype].drop(columns='celltype').sort_values(by=rank_metrics)
    if(rank_metrics=='odds'):
        rnk['odds'] =np.log2(rnk['odds'])
        rnk=rnk.reset_index(drop=True)
        df=rnk.drop(columns='gene')        
        np.random.seed(seed=rseed)
        for i in  rnk.index[(rnk['odds']==np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([np.inf])].max(0)*(1+np.random.uniform()/100))

        for i in  rnk.index[(rnk['odds']==-np.inf).tolist()].tolist():
            rnk.iloc[i,1]= float(df[~df.isin([-np.inf])].min(0)*(1+np.random.uniform()/100))
        #rnk['odds']=rnk['odds']/max(abs(rnk['odds']))
        
    print(rnk.shape)
    print(rnk.head(1))
    print(rnk.tail(1))
    
    gs_res_a = gp.prerank(rnk=rnk, # or data='./P53_resampling_data.txt'
                     gene_sets=glist, # enrichr library names or gmt file
                     #set permutation_type to phenotype if samples >=15
                     permutation_num=nperm, # reduce number to speed up test
                     outdir=None,  # do not write output to disk
                     no_plot=True, # Skip plotting
                     #weighted_score_type=1,
                     #ascending=False,
                        seed=rseed,
                     min_size=1,
                     max_size=5000,
                     processes=8,**kwargs)
                     #format='png')
    return(gs_res_a)

## Run on complete list 

In [4]:
gs_res_a=run_GSEA_gene(celltype='alpha',nperm=5000,glist='../dat/glists/gsea_final_all.gmt')
gs_res_b=run_GSEA_gene(celltype='beta',rseed=2000,nperm=5000,glist='../dat/glists/gsea_final_all.gmt')
gs_res_d=run_GSEA_gene(celltype='delta',nperm=5000,glist='../dat/glists/gsea_final_all.gmt')
display(gs_res_a.res2d.sort_index())
display(gs_res_b.res2d.sort_index())
gs_res_d.res2d.sort_index()

(18771, 2)
   gene      odds
0  MT1G -5.646229
          gene      odds
18770  OSBPL1A  4.822249


2019-11-19 11:17:18,396 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(18655, 2)
      gene      odds
0  PLA2G4E -4.410151
         gene      odds
18654  ZNF541  4.099836


2019-11-19 11:18:02,307 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(16219, 2)
     gene      odds
0  AMOTL1 -4.857858
        gene      odds
16218  ACACA  3.179479


2019-11-19 11:18:45,470 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.861415,3.624232,0.000000,1.000000e+09,645,645,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...
Alpha2,-0.689650,NaN,NaN,1.000000e+00,4508,4508,NFIB;EML1;ALAS1;ADD1;LARP4B;RCL1;DOT1L;OSBP2;P...,DDX11;RNF13;B9D1;MAPK7;DCTN4;ARHGAP10;RP11-176...
Beta sub1_xin,0.539711,1.288704,0.185860,1.580436e-01,13,13,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN;P...,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN
Beta sub2_xin,-0.169407,-0.560414,0.985121,9.785503e-01,28,28,NPY;TFF3;RBP1;STMN2;SCG2;GNAS;ID1;PAM;SEC11C;R...,FOS;JUNB;MLLT11
Beta sub3_xin,0.283955,0.625279,0.879593,9.680355e-01,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS
Beta sub4_xin,-0.385038,-1.988573,0.000000,1.000000e+09,390,381,GDF15;PSMF1;GCG;MMP7;TM4SF4;SERPINA1;PACSIN2;K...,TRAPPC4;PNO1;PTGES3;RGS2;ACP1;EIF5;BUD31;SAT1;...
Beta1,0.734384,3.113303,0.000000,0.000000e+00,856,837,TNS1;SLC51B;EML1;INPP4B;IFI44;ADD1;AMZ1;ZIC1;A...,TNS1;SLC51B;EML1;INPP4B;IFI44;ADD1;AMZ1;ZIC1;A...
Beta2,-0.312826,-1.712968,0.000000,8.676267e-03,671,667,KIAA1217;PKIB;PALLD;MAML3;LTBP1;NRXN3;TNS3;ROB...,ZNF841;BEND3;ATP2A2;TCF12;DAGLB;MPP6;CRYAB;BAR...
Delta1,0.799171,2.444310,0.000000,0.000000e+00,39,39,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.700971,1.927020,0.000000,0.000000e+00,645,638,TECR;RHBDL2;PEX5L;VDR;FADS2;SLC43A3;CPA1;PSMF1...,TECR;RHBDL2;PEX5L;VDR;FADS2;SLC43A3;CPA1;PSMF1...
Alpha2,-0.302455,NaN,NaN,1.000000e+09,4508,4473,TOM1L2;NEDD4L;GRIK4;PGM1;TEX2;TUBGCP4;TECR;PDE...,KIAA1671;CTCF;STMN1;RBKS;BRE;TMED2;SPCS3;TBC1D...
Beta sub1_xin,0.410182,0.852073,0.692288,8.309198e-01,13,13,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;TUBA4A;P...,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A
Beta sub2_xin,0.539887,1.260279,0.167339,1.186039e-01,28,27,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...
Beta sub3_xin,0.789514,1.577676,0.010323,4.476522e-03,13,10,INS;ASB9;DLK1;CHGA;CKB;LAMP1;IGFBP7;TIMP1;CPE;...,INS;ASB9;DLK1
Beta sub4_xin,-0.261620,NaN,NaN,1.000000e+09,390,380,PSMF1;RAB7A;KRT8;ATP6V0D1;MMP7;COPZ1;PFN1;DBP;...,RER1;NOP10;SAT1;EIF1;BTG1;NXT1;TSC22D1;PPIB;UP...
Beta1,0.825824,2.282203,0.000000,0.000000e+00,856,856,TIGIT;PNMAL2;TOM1L2;ZNF541;GRIK4;PRSS53;DKFZP7...,TIGIT;PNMAL2;TOM1L2;ZNF541;GRIK4;PRSS53;DKFZP7...
Beta2,-0.617612,NaN,NaN,1.000000e+09,671,671,NEDD4L;EIF2AK4;FAM160A1;SDK1;RRAS2;NCOA7;ROBO2...,GAREM;VRK2;SCARB2;CEP120;SLC35A5;ATG3;MAP3K5;F...
Delta1,0.731362,1.788877,0.000000,0.000000e+00,39,39,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.605749,3.423117,0.000000,0.000000,645,608,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;ROBO2;NUP35...,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;ROBO2;NUP35...
Alpha2,-0.423467,-1.942539,0.000000,0.000223,4508,4410,CARM1;C20orf195;GPR56;ABCG2;ST5;ATXN7;MPHOSPH9...,ESYT1;PNP;TBC1D10B;TFAP2A;ZNF76;MYLPF;ANXA7;AQ...
Beta sub1_xin,0.518657,1.507206,0.057307,0.055230,13,13,FXYD2;TAGLN2;SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRS...,FXYD2;TAGLN2;SCGN;FFAR4
Beta sub2_xin,-0.487470,-1.449099,0.055468,0.053003,28,25,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;PA...,PCP4;RASD1;ID3;RPS4X;MLLT11
Beta sub3_xin,-0.343159,-0.766675,0.792597,0.896905,13,8,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB;CPE;TIMP1,TIMP1
Beta sub4_xin,-0.305729,-1.329631,0.008145,0.102940,390,376,IFRD1;HLA-E;KRT8;GDF15;SST;MMP7;LRRC59;CHCHD3;...,EIF6;ANXA7;HSPA5;PACSIN2;ARPP19;TIMM17A;DYNLT3...
Beta1,0.554587,3.312039,0.000000,0.000000,856,802,NGEF;SPON2;ADAMTSL3;GPR56;MOB2;ZIC1;MAP3K7CL;R...,NGEF;SPON2;ADAMTSL3;GPR56;MOB2;ZIC1;MAP3K7CL;R...
Beta2,-0.375498,-1.669181,0.000000,0.010025,671,662,ROBO2;MAP4;MTHFD1L;KIAA1217;NHSL1;KCNIP4;FOXP1...,EIF2AK4;DCBLD2;MET;NFIL3;ABHD3;PRDM2;WDR63;FOS...
Delta1,0.946017,3.704180,0.000000,0.000000,39,39,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...


In [11]:
display(gs_res_a.res2d.sort_index())
display(gs_res_b.res2d.sort_index())
gs_res_d.res2d.sort_index()

,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.861415,3.619519,0.000000,0.000000,645,645,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...
Beta sub1_xin,0.539711,1.289142,0.190502,0.172469,13,13,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN;P...,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN
Beta sub2_xin,-0.169407,-0.555646,0.985324,0.979351,28,28,NPY;TFF3;RBP1;STMN2;SCG2;GNAS;ID1;PAM;SEC11C;R...,FOS;JUNB;MLLT11
Beta sub3_xin,0.283955,0.627625,0.876569,0.959068,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS
Beta sub4_xin,-0.385038,-1.990729,0.000000,0.000452,390,381,GDF15;PSMF1;GCG;MMP7;TM4SF4;SERPINA1;PACSIN2;K...,TRAPPC4;PNO1;PTGES3;RGS2;ACP1;EIF5;BUD31;SAT1;...
Beta1,0.734384,3.106538,0.000000,0.000000,856,837,TNS1;SLC51B;EML1;INPP4B;IFI44;ADD1;AMZ1;ZIC1;A...,TNS1;SLC51B;EML1;INPP4B;IFI44;ADD1;AMZ1;ZIC1;A...
Beta2,-0.312826,-1.694721,0.000000,0.017634,671,667,KIAA1217;PKIB;PALLD;MAML3;LTBP1;NRXN3;TNS3;ROB...,ZNF841;BEND3;ATP2A2;TCF12;DAGLB;MPP6;CRYAB;BAR...
Delta1,0.799171,2.439141,0.000000,0.000000,39,39,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...
Mawla_Beta_1,0.504267,1.187817,0.284288,0.245491,14,12,IGSF1;G6PC2;EDN3;ASB9;PCSK1;CECR1;SORL1;KLHDC8...,IGSF1;G6PC2;EDN3


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.700971,1.931945,0.000000,0.000000,645,638,TECR;RHBDL2;PEX5L;VDR;FADS2;SLC43A3;CPA1;PSMF1...,TECR;RHBDL2;PEX5L;VDR;FADS2;SLC43A3;CPA1;PSMF1...
Beta sub1_xin,0.410182,0.861755,0.674205,0.789616,13,13,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;TUBA4A;P...,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A
Beta sub2_xin,0.539887,1.261668,0.155213,0.163180,28,27,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...
Beta sub3_xin,0.789514,1.559100,0.011614,0.009627,13,10,INS;ASB9;DLK1;CHGA;CKB;LAMP1;IGFBP7;TIMP1;CPE;...,INS;ASB9;DLK1
Beta sub4_xin,-0.261690,-1.122904,0.000000,0.577369,390,380,PSMF1;RAB7A;KRT8;ATP6V0D1;MMP7;COPZ1;PFN1;DBP;...,RER1;NOP10;SAT1;EIF1;BTG1;NXT1;TSC22D1;PPIB;UP...
Beta1,0.825825,2.278312,0.000000,0.000000,856,856,PNMAL2;TOM1L2;TIGIT;ZNF541;GRIK4;PRSS53;DKFZP7...,PNMAL2;TOM1L2;TIGIT;ZNF541;GRIK4;PRSS53;DKFZP7...
Beta2,-0.617563,NaN,NaN,1.000000,671,671,NEDD4L;EIF2AK4;FAM160A1;SDK1;RRAS2;NCOA7;ROBO2...,GAREM;VRK2;SCARB2;CEP120;SLC35A5;ATG3;MAP3K5;F...
Delta1,0.731362,1.787061,0.000000,0.000114,39,39,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...
Mawla_Beta_1,0.574737,1.198907,0.246086,0.206683,14,13,ASB9;G6PC2;CECR1;PCSK1;IAPP;KLHDC8A;IGSF1;MAFA...,ASB9;G6PC2;CECR1;PCSK1;IAPP;KLHDC8A


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.605749,3.499603,0.000000,0.000000,645,608,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;ROBO2;NUP35...,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;ROBO2;NUP35...
Beta sub1_xin,0.518657,1.508639,0.057533,0.054751,13,13,FXYD2;TAGLN2;SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRS...,FXYD2;TAGLN2;SCGN;FFAR4
Beta sub2_xin,-0.487470,-1.451656,0.052916,0.097415,28,25,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;PA...,PCP4;RASD1;ID3;RPS4X;MLLT11
Beta sub3_xin,-0.343159,-0.764323,0.798014,0.879116,13,8,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB;CPE;TIMP1,TIMP1
Beta sub4_xin,-0.305729,-1.329330,0.009397,0.158617,390,376,IFRD1;HLA-E;KRT8;GDF15;SST;MMP7;LRRC59;CHCHD3;...,EIF6;ANXA7;HSPA5;PACSIN2;ARPP19;TIMM17A;DYNLT3...
Beta1,0.554587,3.382422,0.000000,0.000000,856,802,NGEF;SPON2;ADAMTSL3;GPR56;MOB2;ZIC1;MAP3K7CL;R...,NGEF;SPON2;ADAMTSL3;GPR56;MOB2;ZIC1;MAP3K7CL;R...
Beta2,-0.375498,-1.670301,0.000000,0.024189,671,662,ROBO2;MAP4;MTHFD1L;KIAA1217;NHSL1;KCNIP4;FOXP1...,EIF2AK4;DCBLD2;MET;NFIL3;ABHD3;PRDM2;WDR63;FOS...
Delta1,0.946017,3.706889,0.000000,0.000000,39,39,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...
Mawla_Beta_1,0.318124,0.883927,0.632163,0.647211,14,11,PCSK1;KLHDC8A;EDN3;CECR1;ASB9;G6PC2;CDKN1C;IGS...,PCSK1;KLHDC8A;EDN3;CECR1


## Run on subset of glists 

### 1. random seed=2000, nperm=10000 for beta list

In [10]:
gs_res_b_sub_2000=run_GSEA_gene(celltype='beta',nperm=10000,rseed=2000,glist='../dat/glists/gsea_final.gmt')
gs_res_b_sub_2000.res2d.sort_index()

,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.699297,1.917962,0.000000,0.000000e+00,500,494,TECR;RHBDL2;PEX5L;FADS2;SLC43A3;CPA1;PSMF1;ZHX...,TECR;RHBDL2;PEX5L;FADS2;SLC43A3;CPA1;PSMF1;ZHX...
Alpha2,-0.327522,NaN,NaN,1.000000e+09,500,498,RPN2;FAM214A;UBR3;MED15;SYNGR4;FAM107B;ZNF587B...,ARRB2;RP11-181C3.1;B3GALNT2;B2M;GOT1;TMEM57;HE...
Beta sub1_xin,0.410182,0.855230,0.688612,1.000000e+00,13,13,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;TUBA4A;P...,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A
Beta sub2_xin,0.539887,1.257678,0.164424,1.990400e-01,28,27,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...
Beta sub3_xin,0.789514,1.570053,0.010321,8.964344e-03,13,10,INS;ASB9;DLK1;CHGA;CKB;LAMP1;IGFBP7;TIMP1;CPE;...,INS;ASB9;DLK1
Beta sub4_xin,-0.261620,NaN,NaN,1.000000e+09,390,380,PSMF1;RAB7A;KRT8;ATP6V0D1;MMP7;COPZ1;PFN1;DBP;...,RER1;NOP10;SAT1;EIF1;BTG1;NXT1;TSC22D1;PPIB;UP...
Beta1,0.808403,2.219413,0.000000,0.000000e+00,500,500,PNMAL2;TOM1L2;PRSS53;DKFZP779J2370;PGM1;CCR8;R...,PNMAL2;TOM1L2;PRSS53;DKFZP779J2370;PGM1;CCR8;R...
Beta2,-0.617948,NaN,NaN,1.000000e+09,500,500,NEDD4L;EIF2AK4;SDK1;RRAS2;NCOA7;PALLD;CELF1;RA...,MEX3B;SPRED1;LONRF3;XKR4;DSTN;AMD1;TNIK;SLC7A1...
Delta1,0.731362,1.784617,0.000109,5.387226e-05,39,39,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...


### 1. random seed=1000, nperm=10000 for all ranking lists & subset genes 

In [6]:
gs_res_a_sub=run_GSEA_gene(celltype='alpha',nperm=10000,glist='../dat/glists/gsea_final.gmt')
gs_res_b_sub=run_GSEA_gene(celltype='beta',nperm=10000,glist='../dat/glists/gsea_final.gmt')
gs_res_d_sub=run_GSEA_gene(celltype='delta',nperm=10000,glist='../dat/glists/gsea_final.gmt')
display(gs_res_a_sub.res2d.sort_index())
display(gs_res_b_sub.res2d.sort_index())
gs_res_d_sub.res2d.sort_index()

(18771, 2)
   gene      odds
0  MT1G -5.646229
          gene      odds
18770  OSBPL1A  4.822249


2019-11-19 11:25:14,626 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(18655, 2)
      gene      odds
0  PLA2G4E -4.440136
         gene      odds
18654  ZNF541  4.099641


2019-11-19 11:28:37,360 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(16219, 2)
     gene      odds
0  AMOTL1 -4.857858
        gene      odds
16218  ACACA  3.179479


2019-11-19 11:31:59,648 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.852683,3.544169,0.000000,0.000000,500,500,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...
Alpha2,-0.549419,-2.901073,0.000000,0.000000,500,500,LLGL2;PLEKHA7;PFAS;DLC1;HIPK2;HNRNPH1;RARA;DFN...,PVRL1;USP25;PANK1;ARNTL;LRTOMT;DLAT;PGBD2;B4GA...
Beta sub1_xin,0.539711,1.289142,0.190502,0.154549,13,13,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN;P...,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN
Beta sub2_xin,-0.169407,-0.555646,0.985324,0.980288,28,28,NPY;TFF3;RBP1;STMN2;SCG2;GNAS;ID1;PAM;SEC11C;R...,FOS;JUNB;MLLT11
Beta sub3_xin,0.283955,0.627625,0.876569,0.966195,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS
Beta sub4_xin,-0.385038,-1.990729,0.000000,0.000302,390,381,GDF15;PSMF1;GCG;MMP7;TM4SF4;SERPINA1;PACSIN2;K...,TRAPPC4;PNO1;PTGES3;RGS2;ACP1;EIF5;BUD31;SAT1;...
Beta1,0.729810,3.035221,0.000000,0.000000,500,489,TNS1;EML1;INPP4B;IFI44;ADD1;ZIC1;ANKRD6;BLK;WW...,TNS1;EML1;INPP4B;IFI44;ADD1;ZIC1;ANKRD6;BLK;WW...
Beta2,-0.335681,-1.790185,0.000000,0.006715,500,497,KIAA1217;PKIB;PALLD;MAML3;LTBP1;NRXN3;TNS3;SOR...,CD109;GLTSCR1L;MTHFD2;NEDD4;STK40;DNTTIP2;IFT1...
Delta1,0.799171,2.439141,0.000000,0.000000,39,39,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.699297,1.921557,0.000000,0.000000e+00,500,494,TECR;RHBDL2;PEX5L;FADS2;SLC43A3;CPA1;PSMF1;ZHX...,TECR;RHBDL2;PEX5L;FADS2;SLC43A3;CPA1;PSMF1;ZHX...
Alpha2,-0.327522,NaN,NaN,1.000000e+00,500,498,RPN2;FAM214A;UBR3;MED15;SYNGR4;FAM107B;ZNF587B...,ARRB2;RP11-181C3.1;B3GALNT2;B2M;GOT1;TMEM57;HE...
Beta sub1_xin,0.410182,0.861755,0.674205,8.940023e-01,13,13,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;TUBA4A;P...,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A
Beta sub2_xin,0.539887,1.261668,0.155213,1.663446e-01,28,27,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...
Beta sub3_xin,0.789514,1.559100,0.011614,1.018099e-02,13,10,INS;ASB9;DLK1;CHGA;CKB;LAMP1;IGFBP7;TIMP1;CPE;...,INS;ASB9;DLK1
Beta sub4_xin,-0.261690,-1.122904,0.000000,1.000000e+09,390,380,PSMF1;RAB7A;KRT8;ATP6V0D1;MMP7;COPZ1;PFN1;DBP;...,RER1;NOP10;SAT1;EIF1;BTG1;NXT1;TSC22D1;PPIB;UP...
Beta1,0.808404,2.216416,0.000000,0.000000e+00,500,500,PNMAL2;TOM1L2;PRSS53;DKFZP779J2370;PGM1;CCR8;R...,PNMAL2;TOM1L2;PRSS53;DKFZP779J2370;PGM1;CCR8;R...
Beta2,-0.617931,NaN,NaN,1.000000e+00,500,500,NEDD4L;EIF2AK4;SDK1;RRAS2;NCOA7;PALLD;CELF1;RA...,MEX3B;SPRED1;LONRF3;XKR4;DSTN;AMD1;TNIK;SLC7A1...
Delta1,0.731362,1.787061,0.000000,1.283048e-04,39,39,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.606671,3.511437,0.000000,0.000000,500,468,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;NUP35;SCML4...,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;NUP35;SCML4...
Alpha2,-0.423847,-1.862444,0.000000,0.000749,500,494,CARM1;ABCG2;PRPF19;HAND1;FXYD6;BAIAP3;TSPAN3;S...,MYO5A;PGBD2;ZNF250;KTI12;CPD;RP11-111M22.2;PDS...
Beta sub1_xin,0.518657,1.508639,0.057533,0.053389,13,13,FXYD2;TAGLN2;SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRS...,FXYD2;TAGLN2;SCGN;FFAR4
Beta sub2_xin,-0.487470,-1.451656,0.052916,0.052518,28,25,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;PA...,PCP4;RASD1;ID3;RPS4X;MLLT11
Beta sub3_xin,-0.343159,-0.764323,0.798014,0.891329,13,8,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB;CPE;TIMP1,TIMP1
Beta sub4_xin,-0.305729,-1.329330,0.009397,0.105231,390,376,IFRD1;HLA-E;KRT8;GDF15;SST;MMP7;LRRC59;CHCHD3;...,EIF6;ANXA7;HSPA5;PACSIN2;ARPP19;TIMM17A;DYNLT3...
Beta1,0.526599,3.038455,0.000000,0.000000,500,466,GPR56;ZIC1;SCML4;GPR160;DNAH9;CCZ1B;RFX4;TENM4...,GPR56;ZIC1;SCML4;GPR160;DNAH9;CCZ1B;RFX4;TENM4...
Beta2,-0.377567,-1.659937,0.000000,0.009708,500,492,MAP4;KIAA1217;NHSL1;FOXP1;RALGDS;GPM6A;BCL9L;P...,SDC2;FAM47E;MAPT;WSB2;RBPJ;RP11-422N16.3;GPX4;...
Delta1,0.946017,3.706889,0.000000,0.000000,39,39,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...


In [5]:
gs_res_a_sub=run_GSEA_gene(celltype='alpha',nperm=10000,glist='../dat/glists/gsea_final.gmt')
gs_res_b_sub=run_GSEA_gene(celltype='beta',nperm=10000,glist='../dat/glists/gsea_final.gmt')
gs_res_d_sub=run_GSEA_gene(celltype='delta',nperm=5000,glist='../dat/glists/gsea_final.gmt')
display(gs_res_a_sub.res2d.sort_index())
display(gs_res_b_sub.res2d.sort_index())
gs_res_d_sub.res2d.sort_index()

(18771, 2)
   gene      odds
0  MT1G -5.646229
          gene      odds
18770  OSBPL1A  4.822249


2019-11-19 11:20:12,789 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(18655, 2)
      gene      odds
0  PLA2G4E -4.410151
         gene      odds
18654  ZNF541  4.099836


2019-11-19 11:21:07,923 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


(16219, 2)
     gene      odds
0  AMOTL1 -4.857858
        gene      odds
16218  ACACA  3.179479


2019-11-19 11:21:57,791 Input gene rankings contains duplicated IDs, Only use the duplicated ID with highest value!


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.852683,3.553162,0.000000,0.000000,500,500,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...,TNS1;SLC51B;ABCA13;WNK2;INPP4B;NHSL2;SLC27A5;H...
Alpha2,-0.549419,-2.893741,0.000000,0.000000,500,500,LLGL2;PLEKHA7;PFAS;DLC1;HIPK2;HNRNPH1;RARA;DFN...,PVRL1;USP25;PANK1;ARNTL;LRTOMT;DLAT;PGBD2;B4GA...
Beta sub1_xin,0.539711,1.288704,0.185860,0.153245,13,13,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN;P...,FXYD2;FFAR4;RBP4;PPP1R1A;SCGB2A1;TAGLN2;SCGN
Beta sub2_xin,-0.169407,-0.560414,0.985121,0.979501,28,28,NPY;TFF3;RBP1;STMN2;SCG2;GNAS;ID1;PAM;SEC11C;R...,FOS;JUNB;MLLT11
Beta sub3_xin,0.283955,0.625279,0.879593,0.967568,13,9,CHGA;INS;ASB9;CPE;IGFBP7;TIMP1;LAMP1;CHGB;CKB,CHGA;INS
Beta sub4_xin,-0.385038,-1.988573,0.000000,0.000202,390,381,GDF15;PSMF1;GCG;MMP7;TM4SF4;SERPINA1;PACSIN2;K...,TRAPPC4;PNO1;PTGES3;RGS2;ACP1;EIF5;BUD31;SAT1;...
Beta1,0.729810,3.037692,0.000000,0.000000,500,489,TNS1;EML1;INPP4B;IFI44;ADD1;ZIC1;ANKRD6;BLK;WW...,TNS1;EML1;INPP4B;IFI44;ADD1;ZIC1;ANKRD6;BLK;WW...
Beta2,-0.335681,-1.795587,0.000000,0.005509,500,497,KIAA1217;PKIB;PALLD;MAML3;LTBP1;NRXN3;TNS3;SOR...,CD109;GLTSCR1L;MTHFD2;NEDD4;STK40;DNTTIP2;IFT1...
Delta1,0.799171,2.444310,0.000000,0.000000,39,39,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...,PDE6C;NCAPG2;CASR;F11;DHRS2;CDHR3;EYS;TMEM72;V...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.699297,1.918882,0.000000,0.000000e+00,500,494,TECR;RHBDL2;PEX5L;FADS2;SLC43A3;CPA1;PSMF1;ZHX...,TECR;RHBDL2;PEX5L;FADS2;SLC43A3;CPA1;PSMF1;ZHX...
Alpha2,-0.327522,NaN,NaN,1.000000e+09,500,498,RPN2;FAM214A;UBR3;MED15;SYNGR4;FAM107B;ZNF587B...,ARRB2;RP11-181C3.1;B3GALNT2;B2M;GOT1;TMEM57;HE...
Beta sub1_xin,0.410182,0.852073,0.692288,1.000000e+00,13,13,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A;TUBA4A;P...,FXYD2;SCGB2A1;FFAR4;RBP4;SCGN;PPP1R1A
Beta sub2_xin,0.539887,1.260279,0.167339,1.983195e-01,28,27,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...,ID1;TFF3;GNAS;RBP1;ID3;IAPP;STMN2;NPY;RPL6;FOS...
Beta sub3_xin,0.789514,1.577676,0.010323,8.057740e-03,13,10,INS;ASB9;DLK1;CHGA;CKB;LAMP1;IGFBP7;TIMP1;CPE;...,INS;ASB9;DLK1
Beta sub4_xin,-0.261620,NaN,NaN,1.000000e+09,390,380,PSMF1;RAB7A;KRT8;ATP6V0D1;MMP7;COPZ1;PFN1;DBP;...,RER1;NOP10;SAT1;EIF1;BTG1;NXT1;TSC22D1;PPIB;UP...
Beta1,0.808403,2.219776,0.000000,0.000000e+00,500,500,PNMAL2;TOM1L2;PRSS53;DKFZP779J2370;PGM1;CCR8;R...,PNMAL2;TOM1L2;PRSS53;DKFZP779J2370;PGM1;CCR8;R...
Beta2,-0.617948,NaN,NaN,1.000000e+09,500,500,NEDD4L;EIF2AK4;SDK1;RRAS2;NCOA7;PALLD;CELF1;RA...,MEX3B;SPRED1;LONRF3;XKR4;DSTN;AMD1;TNIK;SLC7A1...
Delta1,0.731362,1.788877,0.000000,0.000000e+00,39,39,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...,CASR;NCOA7;PDE6C;GJD3;GPM6A;STARD8;RPH3AL;SLC3...


,es,nes,pval,fdr,geneset_size,matched_size,genes,ledge_genes
Term,,,,,,,,
Alpha1,0.606671,3.505240,0.000000,0.000000,500,468,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;NUP35;SCML4...,GPER1;EDAR;GPR56;PRMT8;WDR7;PDZRN4;NUP35;SCML4...
Alpha2,-0.423847,-1.866949,0.000000,0.000673,500,494,CARM1;ABCG2;PRPF19;HAND1;FXYD6;BAIAP3;TSPAN3;S...,MYO5A;PGBD2;ZNF250;KTI12;CPD;RP11-111M22.2;PDS...
Beta sub1_xin,0.518657,1.507206,0.057307,0.053805,13,13,FXYD2;TAGLN2;SCGN;FFAR4;RBP4;ASCL2;SCGB2A1;PRS...,FXYD2;TAGLN2;SCGN;FFAR4
Beta sub2_xin,-0.487470,-1.449099,0.055468,0.053226,28,25,NENF;RBP1;GPX3;STMN2;PEMT;IGFBP5;ID1;SEC11C;PA...,PCP4;RASD1;ID3;RPS4X;MLLT11
Beta sub3_xin,-0.343159,-0.766675,0.792597,0.888763,13,8,CHGA;ASB9;CHGB;LAMP1;IGFBP7;CKB;CPE;TIMP1,TIMP1
Beta sub4_xin,-0.305729,-1.329631,0.008145,0.105974,390,376,IFRD1;HLA-E;KRT8;GDF15;SST;MMP7;LRRC59;CHCHD3;...,EIF6;ANXA7;HSPA5;PACSIN2;ARPP19;TIMM17A;DYNLT3...
Beta1,0.526599,2.989928,0.000000,0.000000,500,466,GPR56;ZIC1;SCML4;GPR160;DNAH9;CCZ1B;RFX4;TENM4...,GPR56;ZIC1;SCML4;GPR160;DNAH9;CCZ1B;RFX4;TENM4...
Beta2,-0.377567,-1.659080,0.000000,0.011064,500,492,MAP4;KIAA1217;NHSL1;FOXP1;RALGDS;GPM6A;BCL9L;P...,SDC2;FAM47E;MAPT;WSB2;RBPJ;RP11-422N16.3;GPX4;...
Delta1,0.946017,3.704180,0.000000,0.000000,39,39,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...,PKD2L2;CTXN3;STARD8;VWC2;NCAPG2;GJD3;CASR;RPH3...


## save data 

In [9]:
gs_res_b.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_beta_res_final.csv')
gs_res_a.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_alpha_res_final.csv')
gs_res_d.res2d[['es','nes','pval','fdr']].to_csv('../figures/Fig2/subfigs/GSEA_delta_res_final.csv')

In [13]:
from gseapy.plot import gseaplot, heatmap
terms = gs_res_b.res2d.index
terms

Index(['Alpha1', 'Beta1', 'Delta1', 'Beta sub3_xin', 'Beta sub2_xin',
       'Mawla_Beta_1', 'multi_correlated', 'Mawla_Beta_2', 'Beta sub1_xin',
       'mutli_anticorrelated', 'Alpha2', 'Beta sub4_xin', 'Beta2', 'Delta2'],
      dtype='object', name='Term')

In [18]:
from gseapy.plot import gseaplot, heatmap
terms = gs_res_b_sub.res2d.index

gseaplot(gs_res_b_sub.ranking, term='mutli_anticorrelated', 
         **gs_res_b_sub.results['mutli_anticorrelated'],ofname='mutli_anticorrelated'+'_beta.pdf')

gseaplot(gs_res_b.ranking, term='mutli_anticorrelated', 
         **gs_res_b.results['mutli_anticorrelated'],ofname='mutli_anticorrelated'+'_beta_all.pdf')

gseaplot(gs_res_b_sub_2000.ranking, term='mutli_anticorrelated', 
         **gs_res_b_sub_2000.results['mutli_anticorrelated'],ofname='mutli_anticorrelated'+'_beta_all.pdf')

